In [1]:
from pandas.io.json import json_normalize
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib.pyplot as plt
import folium,branca
import json
import os
from folium.features import GeoJson, GeoJsonTooltip
import geopandas as gpd

In [4]:
m = folium.Map(location=(47.606209,-122.332069),zoom_start=12)


folium.GeoJson(beat_geo, style_function=lambda feature: {
        'fillColor': 'green' if (feature['properties']['name'] == "C1" or
                                 feature['properties']['name'] == "C2" or\
                                 feature['properties']['name'] == "C3" or\
                                feature['properties']['name'] == "F1" or\
                                feature['properties']['name'] == "F3" or\
                                feature['properties']['name'] == "G1" or\
                                feature['properties']['name'] == "G3" or\
                                feature['properties']['name'] == "N1" or\
                                feature['properties']['name'] == "O2" or\
                                feature['properties']['name'] == "O3" or\
                                feature['properties']['name'] == "Q1" or
                                feature['properties']['name'] == "S2" or
                                feature['properties']['name'] == "W3"  ) 
        else '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(m)
folium.LayerControl().add_to(m)
m.save('test.html')

NameError: name 'beat_geo' is not defined

In [5]:
crime = os.path.join('C:/Users/12370/Desktop/data_science/Crime_Data.csv')
df = pd.read_csv(crime)
df.dropna(inplace = True)
df.head()

,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood
0,20080000465209,12/13/1908,2114.0,12/13/2008,2114.0,DUI,DUI-LIQUOR,EAST,G,G2,CENTRAL AREA/SQUIRE PARK
1,20100000202011,06/15/1964,0.0,06/15/2010,1031.0,FAMILY OFFENSE-NONVIOLENT,CHILD-OTHER,WEST,Q,Q2,QUEEN ANNE
2,2012000023437,01/01/1973,0.0,01/25/2012,1048.0,SEX OFFENSE-OTHER,SEXOFF-OTHER,NORTH,N,N2,NORTHGATE
5,1975000079415,12/16/1975,900.0,12/16/1975,1500.0,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTH,R,R3,LAKEWOOD/SEWARD PARK
9,20090000300201,01/01/1978,1.0,08/25/2009,1016.0,SEX OFFENSE-OTHER,SEXOFF-OTHER,SOUTHWEST,W,W1,ALKI


In [6]:
df_new = df[df['Occurred Date'].str.contains('[0,1][0,9]/[0,3][0,9]/2[0,9]*')]
df_new.head()

,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood
397,20060000462306,10/30/2006,1235.0,10/30/2006,1500.0,SEX OFFENSE-OTHER,SEXOFF-OTHER,SOUTH,O,O1,SODO
631,20070000373571,09/09/2007,1137.0,09/09/2007,1137.0,DUI,DUI-LIQUOR,WEST,Q,Q2,QUEEN ANNE
648,20090000376922,09/30/2007,2000.0,10/26/2009,1755.0,PROSTITUTION,PROSTITUTION-ASSIST-PROMOTE,WEST,K,K1,DOWNTOWN COMMERCIAL
649,201100002424,09/30/2007,1600.0,01/03/2011,420.0,RAPE,SEXOFF-SODOMY,EAST,E,E1,CAPITOL HILL
650,2009000018704,09/30/2007,1530.0,01/15/2009,1820.0,THEFT-ALL OTHER,THEFT-OTH,NORTH,N,N3,NORTHGATE


In [8]:
beat = df_new.Beat.value_counts().sort_index()
beat.head()
beat

B1    123
B2    158
B3    116
C1     90
C2     72
C3     82
D1    140
D2    131
D3    116
E1    101
E2    166
E3    107
F1     66
F2     99
F3     85
G1     64
G2    105
G3     62
J1    101
J2    110
J3     96
K1    102
K2    125
K3    199
L1     94
L2    142
L3    106
M1    153
M2    127
M3    132
N1     77
N2     98
N3    100
O1    106
O2     46
O3     60
Q1     83
Q2    132
Q3    151
R1    110
R2    109
R3    104
S1     94
S2     91
S3     98
U1    167
U2    150
U3    145
W1     92
W2    122
W3     80
Name: Beat, dtype: int64

In [19]:
beat_geo = os.path.join('C:/Users/12370/Desktop/data_science/spd-beats.geojson')

In [9]:
beat.describe()

count     51.000000
mean     109.509804
std       31.093004
min       46.000000
25%       91.500000
50%      105.000000
75%      129.000000
max      199.000000
Name: Beat, dtype: float64

In [23]:
beat_low = beat[beat.values<91.5]

In [8]:
beat_low

C1    90
C2    72
C3    82
F1    66
F3    85
G1    64
G3    62
N1    77
O2    46
O3    60
Q1    83
S2    91
W3    80
Name: Beat, dtype: int64

In [24]:
from branca.colormap import linear

colormap = linear.YlOrRd_09.scale(
    beat.values.min(),
    beat.values.max())
colormap.caption = 'CRIME_REPORT'
colormap


In [25]:
color_dict = {key: colormap(beat[key]) for key in beat.index}

In [45]:
from folium.plugins import MarkerCluster
map_seattle = folium.Map(location=(47.606209,-122.332069),zoom_start=12)

folium.GeoJson(
    beat_geo,
    name='crime_percentage',
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['name']],
        'color': 'white',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.8,
           
    }
).add_to(map_seattle)

folium.GeoJson(beat_geo,
               name='final_beat',
               style_function=lambda feature: {
        'fillColor': 'none' if (
                                 feature['properties']['name'] == "C2"  ) 
        else 'black',
        'color': 'white',
        'weight': 2,
        'dashArray': '3',
     'fillOpacity': 0.8,
    }).add_to(map_seattle)
folium.GeoJson(beat_geo,
               name='safe_beat',
               style_function=lambda feature: {
        'fillColor': 'green' if (feature['properties']['name'] == "C1" or
                                 feature['properties']['name'] == "C2" or\
                                 feature['properties']['name'] == "C3" or\
                                feature['properties']['name'] == "F1" or\
                                feature['properties']['name'] == "F3" or\
                                feature['properties']['name'] == "G1" or\
                                feature['properties']['name'] == "G3" or\
                                feature['properties']['name'] == "N1" or\
                                feature['properties']['name'] == "O2" or\
                                feature['properties']['name'] == "O3" or\
                                feature['properties']['name'] == "Q1" or
                                feature['properties']['name'] == "S2" or
                                feature['properties']['name'] == "W3"  ) 
        else '',
        'color': 'white',
        'weight': 2,
        'dashArray': '3',
   
     'fillOpacity': 0.8
    }).add_to(map_seattle)


# folium.GeoJson(school_geo,
#              name='School'
#               ).add_to(map_seattle)


marker_cluster2 = MarkerCluster(name='school').add_to(map_seattle)
# add markers to map
for lat, lng, locid, name in zip(df_school['Latitude'], df_school['Longitude'], df_school['Status'], df_school['Name']):
    label = '{}, {}'.format(locid, name)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        popup=label,
        icon=folium.Icon(color='blue', icon='info-sign'),
        ).add_to(marker_cluster2)  



marker_cluster = MarkerCluster(name='park').add_to(map_seattle)
# add markers to map
for lat, lng, locid, name in zip(df_park['Latitude'], df_park['Longitude'], df_park['LocID'], df_park['Name']):
    label = '{}, {}'.format(locid, name)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        popup=label,
        icon=folium.Icon(color='green', icon='ok-sign'),
        ).add_to(marker_cluster)  

marker_cluster.add_to(map_seattle)

colormap.add_to(map_seattle)
folium.LayerControl().add_to(map_seattle)

map_seattle.save('test.html')

In [3]:
school_geo = os.path.join('C:/Users/12370/Desktop/data_science/sps_school_site_2015_2016.geojson')

In [36]:
park = os.path.join('C:/Users/12370/Desktop/data_science/Seattle_Parks_And_Recreation_Park_Addresses.csv')
df_park = pd.read_csv(park)
df_park.dtypes


PMAID           int64
LocID           int64
Name           object
Address        object
ZIP Code        int64
X Coord       float64
Y Coord       float64
Location 1     object
dtype: object

In [37]:
df_park.dropna(inplace=True)

In [41]:
df_park.rename(columns={'X Coord':'Longitude','Y Coord':'Latitude'},inplace=True)

In [42]:
df_park.head()

,PMAID,LocID,Name,Address,ZIP Code,Longitude,Latitude,Location 1
0,281,2545,12th and Howe Play Park,1200 W Howe St,98119,-122.372985,47.636097,"(47.636097, -122.372985)"
1,4159,2387,12th Ave S Viewpoint,2821 12TH Ave S,98144,-122.317765,47.577953,"(47.577953, -122.317765)"
2,4467,2382,12th Ave Square Park,564 12th Ave,98122,-122.316455,47.607427,"(47.607427, -122.316455)"
3,4010,2546,14th Ave NW Boat Ramp,4400 14th Ave NW,98107,-122.373536,47.660775,"(47.660775, -122.373536)"
4,296,296,3001 E Madison,3001 E Madison St,98112,-122.293173,47.625169,"(47.625169, -122.293173)"


In [43]:
map_seattle = folium.Map(location=(47.606209,-122.332069),zoom_start=12)

marker_cluster = MarkerCluster(name='park').add_to(map_seattle)
# add markers to map
for lat, lng, locid, name in zip(df_park['Latitude'], df_park['Longitude'], df_park['LocID'], df_park['Name']):
    label = '{}, {}'.format(locid, name)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        popup=label,
        icon=folium.Icon(color='green', icon='ok-sign'),
        ).add_to(marker_cluster)  
map_seattle

In [127]:
lats=[]
lngs=[]
for lat, lng, locid, name in zip(df_park['Latitude'], df_park['Longitude'], df_park['LocID'], df_park['Name']):
    lats.append(lat)
    lngs.append(lng)
    locations = list(zip(lats, lngs))

In [128]:
popups = ['LocID:{}<br>Name:{}'.format(locid, name) for (locid, name) in zip(df_park['LocID'], df_park['Name'])]

In [4]:
with open(school_geo,'r')as f:
    data = json.load(f)


In [8]:
school_data = data['features']
school_data[0]

{'type': 'Feature',
 'id': 1,
 'properties': {'PROPERTY_L': 'Jane Addams', 'Status': 'MS'},
 'geometry': {'type': 'Point',
  'coordinates': [-122.29300902493404, 47.70994486276947]}}

In [7]:
# define the dataframe columns
column_names = ['Name', 'Status', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_school = pd.DataFrame(columns=column_names)
df_school.head()

,Name,Status,Latitude,Longitude


In [15]:
for data in school_data:
    school_name = data['properties']['PROPERTY_L'] 
    school_status = data['properties']['Status']
        
    school_latlon = data['geometry']['coordinates']
    school_lat = school_latlon[1]
    school_lon = school_latlon[0]
    
    df_school = df_school.append({'Name': school_name,
                                          'Status': school_status,
                                          'Latitude':school_lat,
                                          'Longitude': school_lon}, ignore_index=True)
    
df_school.head()

,Name,Status,Latitude,Longitude
0,Jane Addams,MS,47.709945,-122.293009
1,Rainier View,ELEM,47.498863,-122.263172
2,Emerson,ELEM,47.514820,-122.258636
3,Roxhill,ELEM,47.518053,-122.370558
4,South Shore K-8,Option ELEM,47.523743,-122.272009


In [17]:
from folium.plugins import MarkerCluster
map_seattle = folium.Map(location=(47.606209,-122.332069),zoom_start=12)

marker_cluster = MarkerCluster(name='school').add_to(map_seattle)
# add markers to map
for lat, lng, locid, name in zip(df_school['Latitude'], df_school['Longitude'], df_school['Status'], df_school['Name']):
    label = '{}, {}'.format(locid, name)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, lng],
        popup=label,
        icon=folium.Icon(color='blue', icon='info-sign'),
        ).add_to(marker_cluster)  
map_seattle

# target neighborhood

In [218]:
df_new.Neighborhood[df_new.Beat==('C2')].unique()

array(['MONTLAKE/PORTAGE BAY', 'MADISON PARK', 'CENTRAL AREA/SQUIRE PARK',
       'MILLER PARK'], dtype=object)